# Using a D-Wave System




Important Links: 
- [Using a D-Wave System](https://docs.ocean.dwavesys.com/en/latest/overview/dwavesys.html#dwavesys)
- [D-wave Handbook](https://docs.dwavesys.com/docs/latest/doc_getting_started.html)



# Embedding a NOT gate on D-wave
## Formulating the problem as a Binary Quadratic Model(BQM)

Given M binary variables the system try to find a values that minimize the following equation:

\begin{equation}
\sum_i q_i x_i + \sum_{i<j} q_{i,j} x_i x_j
\end{equation}

For this we represent a NOT gate by the following penalty function, where x is the input and z the output:

\begin{equation}
P(x,z) = 2xz−x−z+1
\end{equation}

The penalty function method is used to rule out the following configurations: $0 \rightarrow 0$ and $1 \rightarrow 1$ . The penalty function P have a valid configuration only when $P = 0$.

Now we need to construct a QUBO for this model, for this we only drop the constant term, if there were higher order terms we would need to reformulate the problem as quadratic, in this problem that will not be necessary. So the QUBO for this model is:

\begin{equation}
E(x_1,x_2) = - x_1 - x_2 + 2 x_1 x_2
\end{equation}

So we have that $x_1 = x$ is the input, $x_2 = z$ is the output, and the linear coefficients are $q_1 = q_2 = -1$ and quadratic coefficients are $q_{1,2} = 2$.

Now the only thing that is left to do is the minor embedding for this QUBO configuration. So we will use `EmbeddingComposite` on the `dwave.system` to do this.

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler(endpoint='https://URL_to_my_D-Wave_system/', 
                                          token='ABC-123456789012345678901234567890', 
                                          solver='My_D-Wave_Solver'))

In [ ]:
Q = {('x', 'x'): -1, ('x', 'z'): 2, ('z', 'x'): 0, ('z', 'z'): -1}
response = sampler.sample_qubo(Q, num_reads=5000)
for datum in response.data(['sample', 'energy', 'num_occurrences']):   
   print(datum.sample, "Energy: ", datum.energy, "Occurrences: ", datum.num_occurrences)

# Embedding a AND gate on D-wave
D-wave does not support natively $K_3$ graphs, so you can not implement a AND gate. In order to do this you have to represent as a penalty model:
\begin{equation}
x_1 x_2 \ -2 (x_1 \ + x_2)x_3 + 3 x_3 
\end{equation}

This penalty model can in turn be represented as the QUBO,

\begin{equation}
E(a_i,b_i;x_i) = 3 x_3 + x_1 x_2 - 2 x_1 x_3 - 2 x_2 x_3
\end{equation}

which is a fully connected $K_3$ graph.

So we have that $x_1, x_2$ are the input, $x_3$ is the output, and the linear coefficients are $q_1 = 3$ and quadratic coefficients are $q_{1,2} = 1$, $q_{1,3} = -2$ and $q_{2,3} = -2$. We do the same as we did for the NOT gate.

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler = DWaveSampler(endpoint='https://URL_to_my_D-Wave_system/', token='ABC-123456789012345678901234567890', solver='My_D-Wave_Solver')
sampler_embedded = EmbeddingComposite(sampler)

In [ ]:
Q = {('x1', 'x2'): 1, ('x1', 'z'): -2, ('x2', 'z'): -2, ('z', 'z'): 3}
response = sampler_embedded.sample_qubo(Q, num_reads=5000)
for datum in response.data(['sample', 'energy', 'num_occurrences']):   
   print(datum.sample, "Energy: ", datum.energy, "Occurrences: ", datum.num_occurrences)

# Map Coloring

In [ ]:
import networkx as nx
G = nx.read_adjlist('usa.adj', delimiter = ',')  

In [ ]:
import dwave_networkx as dnx
from hybrid.reference.kerberos import KerberosSampler
coloring = dnx.min_vertex_coloring(G, sampler=KerberosSampler(), chromatic_ub=4, max_iter=10, convergence=3)
print(set(coloring.values()))

In [ ]:
import matplotlib.pyplot as plt
node_colors = [coloring.get(node) for node in G.nodes()]
if dnx.is_vertex_coloring(G, coloring):  # adjust the next line if using a different map
   nx.draw(G, pos=nx.shell_layout(G, nlist = [list(G.nodes)[x:x+10] for x in range(0, 50, 10)] + [[list(G.nodes)[50]]]), with_labels=True, node_color=node_colors, node_size=400, cmap=plt.cm.rainbow)
plt.show()